## Demonstration combining Widgets with Charts

In [1]:
import Graphics.Rendering.Chart
import IHaskell.Display.Widgets
import IHaskell.Display
import Data.Default.Class
import Control.Lens

f = sin              -- The function which we will plot
domain = (0.0, 10.0) -- The domain of the function which we will plot

grain = (snd domain - fst domain) / 100.0

numericIntegral [] = 0
numericIntegral [_] = 0
numericIntegral ((x1,y1):(x2,y2):ps) = 
    ((x2 - x1) * ((y1 + y2) / 2.0)) + numericIntegral ((x2,y2):ps)

sliderRange <- mkFloatRangeSlider
setField sliderRange MinFloat (fst domain)
setField sliderRange MaxFloat (snd domain)
setField sliderRange StepFloat (Just grain)
setField sliderRange FloatPairValue (fst domain, snd domain)

outChart <- mkOutput

drawChart = do 
  (rangeMin, rangeMax) <- getField sliderRange FloatPairValue
  let 
    pts = [(x, f x) | x <- [fst domain, grain .. snd domain]]    
    ptsRange = takeWhile ((<= rangeMax) . fst) $ dropWhile ((< rangeMin) . fst) pts
    layout = def & layout_plots .~
      [ toPlot $ def & plot_lines_values .~ [pts]
      , toPlot $ def & plot_fillbetween_values .~ [(x,(0.0,y)) | (x,y) <- ptsRange]
      ]
  displayCaption <- display $ plain 
      $ "Area under the curve = " <> show (numericIntegral ptsRange)
  displayChart <- display $ toRenderable layout
  setField outChart Outputs [OutputData displayCaption, OutputData displayChart]
      
setField sliderRange ChangeHandler drawChart

sliderRange
outChart
drawChart